In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-07 22:44:37.568627: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 22:44:37.680777: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-07 22:44:38.237021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Generate the Train/Dev/Test

In [3]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
pd.read_csv(file_path)

,Unnamed: 0,id,Main_ID,Secondary_ID,Real,Full_Band_MelGan,HifiGan,MelGan,MelGanLarge,Multi_Band_MelGan,Parallel_WaveGan,Waveglow,ElevenLabs,UberDuck,transcript_1,transcript_2
0,0,LJ001-0001,LJ001,1,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,1,LJ001-0002,LJ001,2,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,in being comparatively modern.,in being comparatively modern.
2,2,LJ001-0003,LJ001,3,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,3,LJ001-0004,LJ001,4,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,4,LJ001-0005,LJ001,5,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13095,13095,LJ050-0274,LJ050,274,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,made certain recommendations which it believes...,made certain recommendations which it believes...
13096,13096,LJ050-0275,LJ050,275,/home/

In [4]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

#data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=source_architectures+['ElevenLabs', 'UberDuck', 'RandWaveFake'])

13055
7833 10444


In [5]:
data_df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ035-0040,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ040-0206,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ010-0191,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ003-0020,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ050-0116,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [6]:
data_df.shape

(143605, 6)

In [7]:
data_df['architecture'].unique()

array(['Real', 'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge',
       'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow', 'ElevenLabs',
       'UberDuck', 'RandWaveFake'], dtype=object)

In [8]:
data_df.to_csv('/home/ubuntu/test.csv')

In [9]:
train_count = data_df[data_df['type'] == 'train'].shape[0]
dev_count = data_df[data_df['type'] == 'dev'].shape[0]
test_count = data_df[data_df['type'] == 'test'].shape[0]

print(f'# of Train instances: {train_count}')
print(f'# of Dev instances: {dev_count}')
print(f'# of Test instances: {test_count}')

# of Train instances: 86163
# of Dev instances: 28721
# of Test instances: 28721


## Generate the Features

In [10]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
feature_df, feature_cols = embedding_manager.generateFeatureDf()

[NeMo I 2023-04-07 22:44:43 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-04-07 22:44:43 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-04-07 22:44:43 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-04-07 22:44:44 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-04-07 22:44:44 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-04-07 22:44:44 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-04-07 22:44:44 features:267] PADDING: 16
[NeMo I 2023-04-07 22:44:45 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [00:03<00:00,  2.83it/s]


In [11]:
#cadence_manager = CadenceModelManager(data_df)
#feature_df, feature_cols =  cadence_manager.run_cadence_feature_extraction_pipeline()

In [12]:
feature_df.head()

,type,id,architecture,path,label,multiclass_label,0,1,2,3,...,182,183,184,185,186,187,188,189,190,191
0,train,LJ035-0040,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.012192,-0.005728,0.010837,0.029937,...,-0.007956,0.002759,-0.015303,-0.022510,0.010036,0.034441,-0.006930,0.014118,0.016982,-0.022488
1,train,LJ040-0206,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.002441,-0.004073,0.012198,0.017363,...,0.005910,0.012659,-0.020849,-0.010093,0.008566,0.034067,0.005381,0.022346,0.009091,-0.013282
2,train,LJ010-0191,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.010807,-0.001423,-0.005432,0.000905,...,-0.022167,0.005136,-0.009735,0.000253,0.009751,0.025495,0.011285,-0.008220,0.002730,0.010948
3,train,LJ003-0020,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.003779,0.010401,0.016487,0.021328,...,0.010385,0.017207,-0.008449,-0.022809,0.002825,0.021302,0.007589,0.015083,0.016631,-0.009994
4,train,LJ050-0116,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,-0.016241,-0.008023,-0.005351,0.029688,...,-0.011631,-0.006752,-0.006677,0.000895,0.009571,0.005901,0.000387,-0.001304,0.009509,-0.008971


## Predict and Evaluate

In [13]:
model_manager = ModelManager('logreg', feature_df, feature_cols, merge_train_dev=True)

In [14]:
acc, logloss = model_manager.trainPredict(label_col='label')

[NeMo W 2023-04-07 22:44:49 nemo_logging:349] /home/ubuntu/MultiModalDeepFake/packages/ModelManager.py:61: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
      self.X_train = self.train[self.feature_cols].copy()
    
[NeMo W 2023-04-07 22:44:50 nemo_logging:349] /home/ubuntu/MultiModalDeepFake/packages/ModelManager.py:68: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
      self.X_test = self.test[self.feature_cols].copy()
    


In [15]:
#model_manager.plotRocCurve()

In [16]:
#model_manager.plotProbaDistribution()

In [17]:
print(acc, logloss)

0.9726275465784433 0.09031276539088326
